# Demo - Create Power BI Reports Quickly in Jupyter Notebooks
The following notebook demonstrates how to quickly create Power BI reports from your data in Jupyter notebooks. 
The notebook will walk through the steps of authenticating to Power BI, importing and processing data, and auto-generating a Power BI report, all within the notebook.

These reports can be used to explore your data, or as a starting point for creating a custom report. Interacting with the report is simple and empowers you to explore your data with just a few simple steps.


In [2]:
from powerbiclient import QuickVisualize, get_dataset_config, Report
from powerbiclient.authentication import DeviceCodeLoginAuthentication

import pandas as pd

### Authenticate to Power BI

#### The following authentication methods can be used to authenticate the user and get the access token:

- [Device flow authentication](https://msal-python.readthedocs.io/en/latest/#msal.PublicClientApplication.acquire_token_by_device_flow)
    
    Code snippet:
    ``` py
  from powerbiclient.authentication import DeviceCodeLoginAuthentication
  device_auth = DeviceCodeLoginAuthentication()
    ```
- [Interactive authentication](https://msal-python.readthedocs.io/en/latest/#msal.PublicClientApplication.acquire_token_interactive)
    
    Code snippet:
    ``` py
  from powerbiclient.authentication import InteractiveLoginAuthentication
  interactive_auth = InteractiveLoginAuthentication()    
    ```

Note: Either use auth object created or access token acquired above while creating the report instance


#### Run the cell below and follow the steps in the output to obtain the auth object using the Device Flow Authentication method for the demo notebook

In [6]:
device_auth = DeviceCodeLoginAuthentication()

Performing device flow authentication. Please follow the instructions below.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code EUDGVHLRU to authenticate.

Device flow authentication successfully completed.
You are now logged in .

The result should be passed only to trusted code in your notebook.


### Import and process data
The code below is used to import a sample dataset as a pandas DataFrame and preform basic preprocessing on it. Make sure you've downloaded the sample csv file, located in the Demo folder

In [7]:
# Import sample CSV as a pandas DataFrame
df = pd.read_excel("Financial Sample.xlsx")

# Display DataFrame
df.head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,Canada,Carretera,NaN,1618.5,3,20,32370.0,0.0,32370.0,16185.0,16185.0,2014-01-01,1,January,2014
1,Government,Germany,Carretera,NaN,1321.0,3,20,26420.0,0.0,26420.0,13210.0,13210.0,2014-01-01,1,January,2014
2,Midmarket,France,Carretera,NaN,2178.0,3,15,32670.0,0.0,32670.0,21780.0,10890.0,2014-06-01,6,June,2014
3,Midmarket,Germany,Carretera,NaN,888.0,3,15,13320.0,0.0,13320.0,8880.0,4440.0,2014-06-01,6,June,2014
4,Midmarket,Mexico,Carretera,NaN,2470.0,3,15,37050.0,0.0,37050.0,24700.0,12350.0,2014-06-01,6,June,2014


In [8]:
# Preprocessing - remove columns, limit to products with over 1000 units sold, and create a binary column indicating whether a product was discounted or not
df = df.drop(['Month Number', 'Month Name', 'Year'], axis=1)
df = df.loc[df['Units Sold'] > 1000]
df = df.fillna('')
df['Discounted'] = df['Discount Band'] != ''

#Display updated DataFrame
df.head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Discounted
0,Government,Canada,Carretera,,1618.5,3,20,32370.0,0.0,32370.0,16185.0,16185.0,2014-01-01,False
1,Government,Germany,Carretera,,1321.0,3,20,26420.0,0.0,26420.0,13210.0,13210.0,2014-01-01,False
2,Midmarket,France,Carretera,,2178.0,3,15,32670.0,0.0,32670.0,21780.0,10890.0,2014-06-01,False
4,Midmarket,Mexico,Carretera,,2470.0,3,15,37050.0,0.0,37050.0,24700.0,12350.0,2014-06-01,False
5,Government,Germany,Carretera,,1513.0,3,350,529550.0,0.0,529550.0,393380.0,136170.0,2014-12-01,False


### Create quick Power BI visualization
Run the code below to use the authentication object and DataFrame obtained above to create a Power BI report and render the report to the notebook.
The output will be an autogenerated report within a few seconds. You can interact with the report, configure the data fields, edit the report layout, and save the report to Power BI.

Learn more here - [Interacting with autogenerated "quick" reports](https://learn.microsoft.com/en-us/power-bi/create-reports/service-interact-quick-report)

To create a new report, simply run the cells below again.

Create a report instance:

In [9]:
# Create a Power BI report from your data
PBI_visualize = QuickVisualize(get_dataset_config(df), auth=device_auth)

### Power BI event handlers

#### Set quick visualization loaded event handler

In [10]:
def loaded_callback(event_details):
    print('Quick visualize has loaded')

PBI_visualize.on('loaded', loaded_callback)

#### Set quick visualization rendered event handler

In [11]:
def rendered_callback(event_details):
    print('Quick visualize has rendered')

PBI_visualize.on('rendered', rendered_callback)

#### Set quick visualization saved event handler

In [12]:
def saved_callback(event_details):
    print('Quick visualize has saved the report')

PBI_visualize.on('saved', saved_callback)

In [51]:
# Render new report
PBI_visualize

QuickVisualize(container_height=400.0, container_width=600.0)

### Configure report size
Run the following code to configure the size of the embedded report output, to ensure the best fit in your notebook.

In [21]:
# Set new width and height for the container
PBI_visualize.set_size(400, 600)

### Load saved report
Once you're happy with your report, you can either keep it in the notebook, or save it to Power BI. The code block below demonstrates how to load the report again using the Power BI library after you've saved it. This can be done as long as the kernel is still active. 
Alternatively, you can load any report using the group_id and report_id parameters, as explained [here](https://github.com/microsoft/powerbi-jupyter/blob/main/DOCUMENTATION.md#__init__-report).

In [ ]:
# Set new access token to the embedded report
report.set_access_token(access_token)

In [22]:
# Set report container's height and width using report object
report.set_size(400, 600)

In [23]:
# Get the list of pages from embedded report
pages = report.get_pages()

In [24]:
pages

[{'name': '42504ed343a8e80b816b',
  'displayName': 'Página 1',
  'isActive': True,
  'visibility': 0,
  'defaultSize': {'type': 0, 'width': 1280, 'height': 720},
  'mobileSize': None,
  'defaultDisplayOption': 0,
  'background': {'color': '#FFFFFF', 'transparency': 100, 'hasImage': False},
  'wallpaper': {'color': '#FFFFFF', 'transparency': 0, 'hasImage': False}}]

In [25]:
# Get the list of visuals on a page
page_name = '42504ed343a8e80b816b'
visuals = report.visuals_on_page(page_name)

In [26]:
visuals

[{'name': '9b204eebe9082f3480d2',
  'title': 'Soma de Units Sold por Ano e Mês',
  'type': 'lineChart',
  'layout': {'x': 20,
   'y': 96,
   'z': 9000,
   'width': 606,
   'height': 458,
   'displayState': {'mode': 0}}},
 {'name': '5b9dfc17190b12dfb05e',
  'title': 'Soma de Discounts por Ano e Mês',
  'type': 'lineChart',
  'layout': {'x': 654,
   'y': 96,
   'z': 7000,
   'width': 606,
   'height': 282,
   'displayState': {'mode': 0}}},
 {'name': 'aba66c18b2c05e25455e',
  'title': 'Soma de COGS por Ano e Mês',
  'type': 'lineChart',
  'layout': {'x': 654,
   'y': 414,
   'z': 6000,
   'width': 289,
   'height': 282,
   'displayState': {'mode': 0}}},
 {'name': '7900eb3e9fdb6abe2218',
  'title': 'Soma de Gross Sales por Ano e Mês',
  'type': 'lineChart',
  'layout': {'x': 971,
   'y': 414,
   'z': 5000,
   'width': 289,
   'height': 282,
   'displayState': {'mode': 0}}},
 {'name': '5dbb3772bc6c59b16d20',
  'title': 'Forma',
  'type': 'shape',
  'layout': {'x': 12,
   'y': 84,
   'z': 10

In [29]:
# Defina a consulta DAX
query = """
EVALUATE
CALCULATE(
    SUM(Sales[Amount]),
    FILTER(Sales, Sales[Region] = "North")
)
"""

In [31]:
report.

AttributeError: 'Report' object has no attribute 'get_dataset_values'

In [34]:
report = PBI_visualize.get_saved_report()

### Load report in output cell

In [35]:
#render report 
report

Report(container_height=400.0, container_width=600.0)

You can interact with this report as part of your notebook. To learn more, visit our [documentation](https://github.com/microsoft/powerbi-jupyter/blob/main/DOCUMENTATION.md#PowerBI-report-embedding-widget) or the report embedding [demo](https://github.com/microsoft/powerbi-jupyter/blob/main/demo/Embed%20Power%20BI%20report%20demo.ipynb).

Argumentos :

group_id string - Opcional. ID do espaço de trabalho do Power BI onde reside o seu relatório. Se o valor não for fornecido, Meu espaço de trabalho será usado.

report_id string - Opcional. ID do relatório do Power BI. Deve ser fornecido para visualizar ou editar um relatório existente.

auth objeto - Opcional. Temos 3 opções de autenticação para incorporar um relatório do Power BI:

Token de acesso (string)
Objeto de autenticação (objeto) - instância de AuthenticationResult (DeviceCodeLoginAuthentication ou InteractiveLoginAuthentication)
Se não for fornecido, o usuário do Power BI será autenticado usando a autenticação Device Flow
view_mode número - Opcional. Modo para incorporar relatório do Power BI (VIEW: 0, EDIT: 1, CREATE: 2). A ser fornecido caso o usuário queira editar ou criar um relatório. (Padrão = VER)

permissions número - Opcional. Permissões necessárias ao incorporar o relatório no modo EDIT. Ignorado no modo VIEW ou CREATE.

Valores para permissões:

READWRITE- Os usuários podem visualizar, editar e salvar o relatório.
COPY- Os usuários podem salvar uma cópia do relatório usando Salvar como.
ALL- Os usuários podem criar, visualizar, editar, salvar e salvar uma cópia do relatório.
dataset_id string - Opcional. Crie um novo relatório usando esse conjunto de dados no espaço de trabalho fornecido do Power BI. Deve ser fornecido para criar um novo relatório a partir de um conjunto de dados existente se report_id não for fornecido.

In [42]:
# report 
# group_id="me" # vá no pbi online em configurações/portal de administração/workspaces/detalhes 
report_id="2f5f4214-6f64-4541-8b9b-41afc4fd7438" # abra o relatório clique em arquivo/inserir relatório/site ou portal/ copie o link e localize o reportID.
report = Report(report_id=report_id, auth=device_auth)
# Alterar a dimensão do report
report.set_size(800, 1600)

In [43]:
report

Report(container_height=800.0, container_width=1600.0)